In [1]:
import numpy as np
import random

### Functions for Mahjong Rules

In [14]:
def initialize_game_state():
    state = np.zeros(3*9+7)
    return state

class Tile:
    def __init__(self, tileno, suit, copyno):
        self.tileno = tileno
        self.suit = suit
        self.copyno = copyno # To ensure that each tile is unique, assign a copy number to the tile (just in case)

class Draw_Pile(object):
    def __init__(self):
        self.deck = []
        for suit in ['Tong','Wan','Suo']:
            for tileno in range(1,10):
                for copyno in range(1,5):
                    tile = Tile(tileno, suit, copyno)
                    self.deck.append(tile)
        # Assign number 0 for the wind and dragon tiles as placeholder
        for suit in ['Dong','Nan','Xi','Bei','Bai_Ban','Fa_Cai','Hong_Zhong']:
            for copyno in range(1,5):
                tile = Tile(0, suit, copyno)
                self.deck.append(tile)
    
    def __len__ (self):
        return len(self.deck)
    
    # Deal a tile from index 0 to the player and removes it from Draw_Pile.deck
    def deal(self):
        if len(self.deck) == 0:
            return None
        else:
            return self.deck.pop(0)

class Discard_Pile(object):
    def __init__(self):
        self.deck = []        
        
class Player(object):
    def __init__(self):
        self.hand = []
        # The fourth piece of a kong is not considered one of the 13 tiles a player must always have in their hand.
        # For every melded kong, hand size increases by 1, so we need to track this
        self.melded_kong_count = 0
        self.melded_hand = []
        
    def discard(self,tile):
        if len(self.hand) == 0:
            return None
        else:
            return self.hand.pop(tile)

def score_hand(hand, test_mode = 0):
    '''score_hand Function takes a hand matrix (structure speified in project description)
        And will return a list of (bool, score):
        If input is a valid hand: (True, multipler (int))
        If input is NOT a valid hand: (False, 0)
        Scoring rules ref: https://en.wikipedia.org/wiki/Hong_Kong_mahjong_scoring_rules
        RMK: no Kong and any wining hand with Kong is NOT considered
    ''' 
    sum_of_hand = np.sum(hand, axis = 1)%10
    sum_of_hand_copy = sum_of_hand.copy()
    wining = (None,0) # will return a list of (bool, score)
    
    # Check Invalid hand
    if any(sum_of_hand > 4) or any(sum_of_hand < 0) or np.sum(sum_of_hand) != 14 or np.any(hand > 10):
        print(hand)
        raise Exception("score_hand funciton error: copy_of_hand out of range")
    
    # Thirteen Orphans
    if np.all(sum_of_hand[[0,8,9,17,18,26]] >= 1) and np.all(sum_of_hand[27:] >= 1):
        if test_mode: print('Wining Hand: Thirteen Orphans (13)')
        return (True, 13)
    
    # Nine Gates
    if (sum_of_hand_copy[9] >= 3 and sum_of_hand_copy[17] >= 3 and np.all(sum_of_hand_copy[10:17] >= 1)) or (sum_of_hand_copy[0] >= 3 and sum_of_hand_copy[8] >= 3 and np.all(sum_of_hand_copy[1:8] >= 1)) or (sum_of_hand_copy[18] >= 3 and sum_of_hand_copy[26] >= 3 and np.all(sum_of_hand_copy[19:26] >= 1)):
        if test_mode:print('wining Hand: Nine Gates (10)')
        return (True, 10)
    
    # Find singular Honor tiles
    if any(sum_of_hand[27:] == 1) or any(sum_of_hand[27:] == 4):
        if test_mode: print('singular or quad honor tiles')
        return (0,0)
    
    # Find Pong count and pairs
    pong_count = np.zeros([5,1], dtype = int)
    chow_count = np.zeros([5,1], dtype = int)
    pair_flag = np.zeros([5,1], dtype = int)
    idx_triplets = np.argwhere(sum_of_hand == 3)[:,0]
    idx_pairs = np.argwhere(sum_of_hand == 2)[:,0]
    idx_quad = np.argwhere(sum_of_hand == 4)[:,0]
    for idx in idx_triplets:
        if idx >=0 and idx <= 8:
            pong_count[0] += 1
        elif idx >= 9 and idx <= 17:
            pong_count[1] += 1
        elif idx >= 18 and idx <= 26:
            pong_count[2] += 1
        elif idx >= 27 and idx <= 30:
            pong_count[3] += 1
        elif idx >= 31 and idx <= 33:
            pong_count[4] += 1
        sum_of_hand[idx] -= 3
        
    if not np.all(pair_flag, axis = 0):
        for idx in np.hstack((np.flip(idx_pairs), idx_quad)):
            if idx >= 31 and idx <= 33:
                pair_flag[4] += 1
            elif idx >= 27 and idx <= 30:
                    pair_flag[3] += 1
            elif idx >=0 and idx <= 8:
                pair_flag[0] += 1
            elif idx >= 9 and idx <= 17:
                pair_flag[1] += 1
            elif idx >= 18 and idx <= 26:
                pair_flag[2] += 1
            sum_of_hand[idx] -= 2
            break
    for idx in idx_quad:
        if idx >=0 and idx <= 8:
            pong_count[0] += 1
        elif idx >= 9 and idx <= 17:
            pong_count[1] += 1
        elif idx >= 18 and idx <= 26:
            pong_count[2] += 1
        elif idx >= 27 and idx <= 30:
            pong_count[3] += 1
        elif idx >= 31 and idx <= 33:
            pong_count[4] += 1
        sum_of_hand[idx] -= 3
    # Test Purpose
    if test_mode: print('Pong Count:{}, Pair Suit:{}'.format(np.sum(pong_count), np.sum(pair_flag)))
                 
    for suit, start, stop in zip([0,1,2], [0,9,18],[9,18,27]):

        sum_of_hand_slice = sum_of_hand[start:stop]
        singular_tileno = np.argwhere(sum_of_hand_slice >= 1)[:,0]
        for i in singular_tileno:
            while sum_of_hand[start + i] > 0:
                if np.all(sum_of_hand[start:stop] == 0):
                    break
                if sum_of_hand[start + i] != 0:
                    if i == 0:
                        if (sum_of_hand_slice[1] == 0 or sum_of_hand_slice[2] == 0):
                            if test_mode: print('Singluar tiles at 1 Suit:{}'.format(suit))
                            return (0,0)
                    elif i == 1:
                        if (sum_of_hand_slice[0] == 0 or sum_of_hand_slice[2] == 0) and (sum_of_hand_slice[2] == 0 or sum_of_hand_slice[3] == 0):
                            if test_mode: print('Singluar tiles at 2 Suit:{}'.format(suit))
                            return (0,0)
                    elif i == 8 :
                        if (sum_of_hand_slice[-2] == 0 or sum_of_hand_slice[-3] == 0):
                            if test_mode: print('Singluar tiles at 9 Suit:{}'.format(suit))
                            return (0,0)
                    elif i == 7:
                        if ((sum_of_hand_slice[-1] == 0 or sum_of_hand_slice[-3] == 0) and (sum_of_hand_slice[-3] == 0 or sum_of_hand_slice[-4] == 0)):
                            if test_mode: print('Singluar tiles at 8 Suit:{}'.format(suit))
                            return (0,0)
                    else: 
                        if(sum_of_hand_slice[i-1] == 0 or sum_of_hand_slice[i+1] == 0) and (sum_of_hand_slice[i-1] == 0 or sum_of_hand_slice[i-2] == 0) and (sum_of_hand_slice[i+1] == 0 or sum_of_hand_slice[i+2] == 0):
                            if test_mode: print('Singluar tiles at {} Suit{}'.format(i, suit))
                            return (0,0)
                    # Form chow meld
                    if i == 8:
                        sum_of_hand[start+i-2:start+i+1] -= 1
                        chow_count[suit] += 1
                    else:           
                        sum_of_hand[start+i:start+i+3] -= 1
                        chow_count[suit] += 1

                    
    sum_of_hand_count = np.bincount(sum_of_hand, minlength = 5)
            
    # Find 2 or more pairs
    if wining[0] is None and sum_of_hand_count[2] > 0:
        if test_mode: print('Existing remaining pairs')
        return (0,0)
    if np.sum((pong_count + chow_count), axis = 0) != 4 and not np.any(pair_flag == 1):
        if test_mode: print('Non a hand')
        return (0,0)
            
#     # Seven Pairs (NON-Traditional)
#     if sum_of_hand_count[2] == 7:
#         if test: print('Wining Hand: Seven Pairs (4)')
#         return (True, 4)
    
    # Extra Points of Triplets of Dragon Tiles
    extra_points = pong_count[4,0].copy()
    if test_mode: print('Triplets of Dragon Tiles: {}'.format(extra_points))
    # All in Triplets
    if np.sum(pong_count, axis = 0) == 4:
        extra_points += 3
        if test_mode: print('All In Triplets: {}'.format(extra_points))

    # Great Winds
    if pong_count[3] == 4 and np.any(pair_flag == 1):
            # All Honor Tiles
            if np.argwhere(pair_flag == 1)[:,0] > 2:
                if test_mode:print('Wining Hand: Great Winds + All Honor Tiles (23)')
                extra_points_allhonor = 10
            else:
                extra_points_allhonor = 0
                if test_mode: print('Wining Hand: Great Winds(13)')
            return (True, 13+ extra_points_allhonor)           
    
    
    # Orphans
    orphan_bincount = np.bincount(sum_of_hand_copy[[0,8,9,17,18,26]], minlength = 4)
    if orphan_bincount[3] == 4 and orphan_bincount[2] == 1:
        if test_mode:print('Wining Hand: Orpahns')
        return (True,10)
    
    # Small Wind
    if pong_count[3] == 3 and pair_flag[3] == 1:
        if pong_count[4] == 1:
            if test_mode:print('Wining Hand: All Honor Tiles + Small Wind (16)')
            return (True, 16)
        elif any(sum_of_hand_copy[0:27] == 3):
            if test_mode:print('Wining Hand: All in Triplets + Small Wind (9)')
            return (True, 9)
        elif np.all(sum_of_hand_copy[31:] == 0):
            if np.all(sum_of_hand_copy[0:18] == 0) and np.all(np.diff(np.argwhere(sum_of_hand_copy[18:27] == 1)[:,0]) == 1):
                if test_mode:print('Wining Hand: Mixed One Suit (Suo) + Small Wind (9)')
                return(True, 9)
            elif np.all(sum_of_hand_copy[9:27] == 0) and np.all(np.diff(np.argwhere(sum_of_hand_copy[0:9] == 1)[:,0]) == 1):
                if test_mode:print('Wining Hand: Mixed One Suit(Tong) + Small Wind (9)')
                return(True, 9)
            elif np.all(sum_of_hand_copy[0:9] == 0) and np.all(sum_of_hand_copy[18:27] == 0) and np.all(np.diff(np.argwhere(sum_of_hand_copy[9:18] == 1)[:,0]) == 1):
                if test_mode:print('Wining Hand: Mixed One Suit(Wan) + Small Wind (9)')
                return(True, 9)     
    
    # Great Dragons
    if pong_count[4] == 3:
        if pong_count[3] == 1 and pair_flag[3] == 1:
            if test_mode:print('Wining Hand: Great Dragons + All Honors (18)')
            return (True, 18)
        elif np.any(np.sum((pong_count[0:4] + chow_count[0:4] + pair_flag[0:4]), axis = 1) == 2):
            if test_mode:print('Wining Hand: Mixed One Suit + Great Dragons (11)')
            return (True, 11 + extra_points)
        elif np.sum((pong_count[0:4] + chow_count[0:4]), axis = 0) == 1 and np.any(pair_flag == 1):
            if test_mode:print('Wining Hand:Great Dragons (8)')
            return (True, 8 + extra_points)
    
    # Small Dragons
    if pong_count[4] == 2 and pair_flag[4] == 1:
        if pong_count[3] == 2:
            if test_mode:print('Wining Hand: All Honor Tiles + Small Dragons (15)')
            return(True, 15)
        elif np.any(pong_count[0:3] == 2) or np.any(chow_count[0:3] == 2) or np.any((pong_count[0:3] + chow_count[0:3]) == 2):
            if test_mode:print('Wining Hand: Mixed One Suit + Small Dragons (8)')
            return (True, 8 + extra_points)
        elif np.sum(pong_count[0:3] + chow_count[0:3], axis = 0) == 2:
            if test_mode:print('Wining Hand: Small Dragons (5)')
            return (True, 5 + extra_points)
        
    # All in one suit
    if np.any((pong_count[0:3] + chow_count[0:3] + pair_flag[0:3]) == 5):
        if test_mode: print('All In One Suit: {}'.format(7))
        return (True, 7 + extra_points)
                
    # All Honor Tiles
    if np.sum(pong_count[3:], axis = 0) == 4 and np.argwhere(pair_flag[3:] == 1)[:,0] > 2:
        if test_mode:print('Wining Hand: All Honor Tiles {}'.format(10+extra_points))
        return (True, 10+extra_points)
    
    # Mixed One Suit
    honor_count = np.sum((pong_count[3:] + pair_flag[3:]), axis = 0)
    if np.any((np.sum((pong_count[0:3] + chow_count[0:3] + pair_flag[0:3]), axis=1) + honor_count) == 5):
        if test_mode:print('Wining Hand: Mixed One Suit {}'.format(3+extra_points))
        return (True, 3+extra_points)
    # Common Hand
    if np.sum(chow_count, axis = 0) == 4 and np.any(pair_flag == 1):
        if test_mode: print('Wining Hand:Common hand')
        return (True, 1+extra_points)
    
    if np.sum([chow_count + pong_count, axis = 0) == 4 and np.any(pair_flag == 1):
        if test_mode: print('Wining Hand:Common hand')
        return (True, 0+extra_points)
    
    return (0, 0)

### Mahjong State Dictionary

In [5]:
mahjong_dict = {0:'1 Tong',1:'2 Tong',2:'3 Tong',3:'4 Tong',4:'5 Tong',5:'6 Tong',6:'7 Tong',7:'8 Tong',8:'9 Tong',
              9:'1 Wan',10:'2 Wan',11:'3 Wan',12:'4 Wan',13:'5 Wan',14:'6 Wan',15:'7 Wan',16:'8 Wan',17:'9 Wan',
              18:'1 Suo',19:'2 Suo',20:'3 Suo',21:'4 Suo',22:'5 Suo',23:'6 Suo',24:'7 Suo',25:'8 Suo',26:'9 Suo',
              27:'Dong',28:'Nan',29:'Xi',30:'Bei',
              31:'Bai_Ban',32:'Fa_Cai',33:'Hong_Zhong'}

In [42]:
random_seed = random.choices(np.arange(33*4),k=14)
state = np.zeros([33,4])
for i in random_seed:
    # int(i/4) gives tile no., i%4 gives copy no.
    state[int(i/4),i%4]=1
state

array([[1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 1.]])

### Functions to View State

In [ ]:
# Use this function to output the tiles to see 
def display_tiles(pile):
    print('tile count =',len(pile))
    for tile in pile:
        print(tile.tileno, tile.suit)

### Functions for Decision Making

In [8]:
def decide(player_hand, drawn_tile, discard_pile):
    #### The decision function to eventually go here
    if winning_hand == 1:
        return score
    elif kong == 1:
        kong_count = kong_count + 1
        return kong_count
    else:
        return tile_to_discard
    
def value_function():
    #### If our value function is to differ from the official scoring rules
    value = score_hand()
    return value

In [4]:
a = np.matrix([[1],[1],[3],[2],[2],[4]])


In [5]:
np.argwhere(a == 1)[1,0]

1

In [6]:
a[2]

matrix([[3]])

In [15]:
all(0 in a, 5 in a)

TypeError: all() takes exactly one argument (2 given)

In [8]:
kk = []
for i in kk:
    print('test')

In [14]:
for j,k in zip([0,9,18], [9,18,27]):
    print(k)

9
18
27


In [3]:
wining = (None,0.0)

In [7]:
type(wining[0]) 


NoneType

In [16]:
a = np.arange(5)

In [28]:
a = np.ones([4,6])
b = np.sum(a, axis = 1)


In [23]:
all(np.diff(a) == 0)

False

In [27]:
a = np.matrix([[0], [1],[1],[1],[0],[0]])

In [23]:
np.hstack((np.argwhere(a == 1)[:,0],np.argwhere(a == 1)[:,0]))

array([1, 2, 3, 1, 2, 3], dtype=int64)

In [29]:
a[2:4] -=2

In [30]:
a

matrix([[ 0],
        [ 1],
        [-1],
        [-1],
        [ 0],
        [ 0]])

In [24]:
a = [1,4]


In [26]:
a.index(4)

1

In [13]:
a = np.matrix([0, 0])

In [14]:
if not np.all(a):
    print('yo')

yo
